In [1]:
import pandas as pd

from carla import OnlineCatalog, MLModelCatalog
from carla.recourse_methods import GrowingSpheres, Wachter, Dice, CCHVAE, Revise
from carla import Benchmark
from carla.evaluation.catalog import SuccessRate, Distance, Redundancy, AvgTime, ConstraintViolation, YNN

/Users/suzie/CARLA/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


/Users/suzie/CARLA/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1. Dataset and model importation

In [2]:
dataset = OnlineCatalog("adult")

In [3]:
model = MLModelCatalog(dataset, "ann", backend="pytorch")

2. Generate a Counterfactual

In [4]:
dice = Dice(model, {})
factuals = dataset.df.sample(1)
cf_dice = dice.get_counterfactuals(factuals)

  0%|          | 0/1 [00:00<?, ?it/s]/Users/suzie/CARLA/.venv/lib/python3.9/site-packages/torch/nn/modules/module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/Users/suzie/CARLA/.venv/lib/python3.9/site-packages/dice_ml/explainer_interfaces/dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  candidate_cfs.at[k, selected_features[k][0]] = random_instances.at[k, selected_features[k][0]]
/Users/suzie/CARLA/.venv/lib/python3.9/site-packages/dice_ml/explainer_interfaces/dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.0' has dtype incompatible with float64, please explicitly c

In [8]:
factuals_decoded = dataset.inverse_transform(factuals)
cf_dice_decoded = dataset.inverse_transform(cf_dice)
factual_row = factuals_decoded.iloc[0]
cf_row = cf_dice_decoded.iloc[0]

df_pair = pd.DataFrame({
    "Factual": factual_row,
    "Counterfactual": cf_row
})

def highlight_diff(row):
    if row.iloc[-1] != row.iloc[-2]:
        return ['background-color: blue'] * len(row)
    else:
        return [''] * len(row)

styled_df = df_pair.style.apply(highlight_diff, axis=1)


styled_df


,Factual,Counterfactual
age,49.000000,49.000000
capital-gain,0.000000,0.000000
capital-loss,0.000000,0.000000
education-num,15.000000,15.000000
fnlwgt,218835.000000,102362.643622
hours-per-week,60.000000,60.000000
income,1,nan
marital-status,Married,Married
native-country,US,US
occupation,Managerial-Specialist,Managerial-Specialist


3. Evaluate the generate cf: 

In [9]:
benchmark_dice = Benchmark(model, dice, factuals, cf_dice)
benchmark_dice.run_benchmark(measures=[SuccessRate(), Distance(model), YNN(model, {"y": 5, "cf_label": 1}),  Redundancy(model, {"cf_label": 1}),  ConstraintViolation(model), AvgTime(dice)])

/Users/suzie/CARLA/carla/models/api/mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  warnings.warn(
/Users/suzie/CARLA/.venv/lib/python3.9/site-packages/torch/nn/modules/module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/Users/suzie/CARLA/carla/models/api/mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  warnings.warn(
/Users/suzie/CARLA/.venv/lib/python3.9/site-packages/torch/nn/modules/module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/Users/suzie/CARLA/carla/models/api/mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  warnings.warn(
/Users/suzie/CARLA/.venv/lib/pytho

,Success_Rate,L0_distance,L1_distance,L2_distance,Linf_distance,y-Nearest-Neighbours,Redundancy,Constraint_Violation,avg_time
0,1.0,1.0,0.078798,0.006209,0.078798,1.0,1.0,1.0,0.224565


Benchmark de plusieurs méthodes: 

In [1]:
import pandas as pd
from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.recourse_methods import GrowingSpheres, Wachter, Dice, CCHVAE, Revise
import carla.evaluation.catalog as evaluation_catalog
from carla.evaluation import Benchmark

dataset = OnlineCatalog("adult")  # ou "compas", "give_me_some_credit"
mlmodel = MLModelCatalog(dataset, "ann", backend="pytorch")  # ici ANN Pytorch

config_cchvae = {
    "data_name": "adult",
    "n_search_samples": 100,
    "p_norm": 1,
    "step": 0.1,
    "max_iter": 1000,
    "binary_cat_features": True,
    "vae_params": {
        "layers": [sum(mlmodel.get_mutable_mask()), 512, 256, 8],
        "train": True,
        "lambda_reg": 0.000001,
        "epochs": 5,
        "lr": 0.001,
        "batch_size": 32
    }
}

config_revise = {
    "data_name": "adult",
    "lambda": 0.5,
    "optimizer": "adam",
    "lr": 0.1,
    "max_iter": 1500,
    "target_class": [0, 1],
    "binary_cat_features": True,
    "vae_params": {
        "layers": [sum(mlmodel.get_mutable_mask()), 512, 256, 8],
        "activFun": None,
        "train": True,
        "lambda_reg": 0.000001,
        "epochs": 5,
        "lr": 0.001,
        "batch_size": 32
    }
}


recourse_methods = {
    "GrowingSpheres": GrowingSpheres(mlmodel),
    #"Wachter": Wachter(mlmodel),
    "Dice": Dice(mlmodel),
    #"CCHVAE": CCHVAE(mlmodel, config_cchvae),
    #"Revise": Revise(mlmodel, dataset, config_revise),
}
n_instances = 10
factuals = dataset.df.sample(10)



all_results = []

for name, recourse_method in recourse_methods.items():
    benchmark = Benchmark(mlmodel, recourse_method, factuals)
    metrics = [
    evaluation_catalog.YNN(mlmodel),
    evaluation_catalog.Distance(mlmodel),
    evaluation_catalog.SuccessRate(),
    evaluation_catalog.Redundancy(mlmodel),
    evaluation_catalog.ConstraintViolation(mlmodel),
    evaluation_catalog.AvgTime(recourse_method),  
    ]
    result = benchmark.run_benchmark(metrics)
    result["Recourse_Method"] = name
    all_results.append(result)

results_df = pd.concat(all_results, ignore_index=True)
results_df.set_index("Recourse_Method", inplace=True)
results_df


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


c:\sgrondin\Explicability\CARLA\CARLA\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[WARNING] From c:\sgrondin\Explicability\CARLA\CARLA\carla\recourse_methods\autoencoder\models\autoencoder.py:12: The name tf.disable_eager_execution is deprecated. Please use tf.compat.v1.disable_eager_execution instead.
 [module_wrapper.py _tfmw_add_deprecation_warning]


c:\sgrondin\Explicability\CARLA\CARLA\carla\models\api\mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  warnings.warn(
c:\sgrondin\Explicability\CARLA\CARLA\.venv\lib\site-packages\torch\nn\modules\module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
c:\sgrondin\Explicability\CARLA\CARLA\carla\models\api\mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  warnings.warn(
c:\sgrondin\Explicability\CARLA\CARLA\.venv\lib\site-packages\torch\nn\modules\module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
c:\sgrondin\Explicability\CARLA\CARLA\carla\models\api\mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: 

,y-Nearest-Neighbours,L0_distance,L1_distance,L2_distance,Linf_distance,Success_Rate,Redundancy,Constraint_Violation,avg_time
Recourse_Method,,,,,,,,,
GrowingSpheres,0.266667,5.777778,1.024311,0.797980,0.699460,0.9,3.666667,0.0,0.003263
Dice,0.600000,1.700000,1.085207,0.882411,0.767117,1.0,0.800000,1.1,0.137430


In [2]:
import pandas as pd
from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.recourse_methods import GrowingSpheres, Wachter, Dice, CCHVAE, Revise
import carla.evaluation.catalog as evaluation_catalog
from carla.evaluation import Benchmark

dataset = OnlineCatalog("adult")  # ou "compas", "give_me_some_credit"
mlmodel = MLModelCatalog(dataset, "ann", backend="pytorch")  # ici ANN Pytorch

config_cchvae = {
    "data_name": "adult",
    "n_search_samples": 100,
    "p_norm": 1,
    "step": 0.1,
    "max_iter": 1000,
    "binary_cat_features": True,
    "vae_params": {
        "layers": [sum(mlmodel.get_mutable_mask()), 512, 256, 8],
        "train": True,
        "lambda_reg": 0.000001,
        "epochs": 5,
        "lr": 0.001,
        "batch_size": 32
    }
}

config_revise = {
    "data_name": "adult",
    "lambda": 0.5,
    "optimizer": "adam",
    "lr": 0.1,
    "max_iter": 1500,
    "target_class": [0, 1],
    "binary_cat_features": True,
    "vae_params": {
        "layers": [sum(mlmodel.get_mutable_mask()), 512, 256, 8],
        "activFun": None,
        "train": True,
        "lambda_reg": 0.000001,
        "epochs": 5,
        "lr": 0.001,
        "batch_size": 32
    }
}


recourse_methods = {
    "GrowingSpheres": GrowingSpheres(mlmodel),
    "Wachter": Wachter(mlmodel, {}),
    "Dice": Dice(mlmodel),
    "CCHVAE": CCHVAE(mlmodel, config_cchvae),
    "Revise": Revise(mlmodel, dataset, config_revise),
}
n_instances = 10
factuals = dataset.df.sample(10)



all_results = []

for name, recourse_method in recourse_methods.items():
    benchmark = Benchmark(mlmodel, recourse_method, factuals)
    metrics = [
    evaluation_catalog.YNN(mlmodel, {"y": 5, "cf_label": 1}),
    evaluation_catalog.Distance(mlmodel),
    evaluation_catalog.SuccessRate(),
    evaluation_catalog.Redundancy(mlmodel, {"cf_label": 1}),
    evaluation_catalog.ConstraintViolation(mlmodel),
    evaluation_catalog.AvgTime(recourse_method),  
    ]
    result = benchmark.run_benchmark(metrics)
    result["Recourse_Method"] = name
    all_results.append(result)

results_df = pd.concat(all_results, ignore_index=True)
results_df.set_index("Recourse_Method", inplace=True)
results_df


[INFO] Start training of Variational Autoencoder... [vae.py fit]
[INFO] [Epoch: 0/5] [objective: 76.177] [vae.py fit]
[INFO] [ELBO train: 76.18] [vae.py fit]
[INFO] [ELBO train: 87.38] [vae.py fit]
[INFO] [ELBO train: 94.29] [vae.py fit]
[INFO] [ELBO train: 101.42] [vae.py fit]
[INFO] [ELBO train: 108.07] [vae.py fit]
[INFO] ... finished training of Variational Autoencoder. [vae.py fit]
[INFO] Start training of Variational Autoencoder... [vae.py fit]
[INFO] [Epoch: 0/5] [objective: 76.525] [vae.py fit]
[INFO] [ELBO train: 76.52] [vae.py fit]
[INFO] [ELBO train: 87.36] [vae.py fit]
[INFO] [ELBO train: 94.79] [vae.py fit]
[INFO] [ELBO train: 101.64] [vae.py fit]
[INFO] [ELBO train: 108.02] [vae.py fit]
[INFO] ... finished training of Variational Autoencoder. [vae.py fit]


/Users/suzie/CARLA/carla/models/api/mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  warnings.warn(
/Users/suzie/CARLA/.venv/lib/python3.9/site-packages/torch/nn/modules/module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/Users/suzie/CARLA/carla/models/api/mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  warnings.warn(
/Users/suzie/CARLA/.venv/lib/python3.9/site-packages/torch/nn/modules/module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/Users/suzie/CARLA/carla/models/api/mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  warnings.warn(
/Users/suzie/CARLA/.venv/lib/pytho

[INFO] Timeout - No Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Timeout - No Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Timeout - No Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Timeout - No Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Timeout - No Counterfactual Explanation Found [wachter.py wachter_recourse]


/Users/suzie/CARLA/.venv/lib/python3.9/site-packages/torch/nn/modules/module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/Users/suzie/CARLA/carla/models/api/mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  warnings.warn(
/Users/suzie/CARLA/.venv/lib/python3.9/site-packages/torch/nn/modules/module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/Users/suzie/CARLA/carla/models/api/mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  warnings.warn(
/Users/suzie/CARLA/.venv/lib/python3.9/site-packages/torch/nn/modules/module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X

[INFO] Counterfactual found! [model.py _counterfactual_optimization]
[INFO] No counterfactual found [model.py _counterfactual_optimization]
[INFO] Counterfactual found! [model.py _counterfactual_optimization]
[INFO] Counterfactual found! [model.py _counterfactual_optimization]
[INFO] Counterfactual found! [model.py _counterfactual_optimization]
[INFO] Counterfactual found! [model.py _counterfactual_optimization]
[INFO] No counterfactual found [model.py _counterfactual_optimization]
[INFO] No counterfactual found [model.py _counterfactual_optimization]
[INFO] Counterfactual found! [model.py _counterfactual_optimization]
[INFO] Counterfactual found! [model.py _counterfactual_optimization]


/Users/suzie/CARLA/.venv/lib/python3.9/site-packages/torch/nn/modules/module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/Users/suzie/CARLA/carla/models/api/mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  warnings.warn(
/Users/suzie/CARLA/.venv/lib/python3.9/site-packages/torch/nn/modules/module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/Users/suzie/CARLA/carla/models/api/mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  warnings.warn(
/Users/suzie/CARLA/.venv/lib/python3.9/site-packages/torch/nn/modules/module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X

,y-Nearest-Neighbours,L0_distance,L1_distance,L2_distance,Linf_distance,Success_Rate,Redundancy,Constraint_Violation,avg_time
Recourse_Method,,,,,,,,,
GrowingSpheres,0.175000,5.750000,1.027800,0.781347,0.577446,0.8,3.500000,0.0,0.004517
Wachter,0.520000,3.600000,0.154455,0.007162,0.025767,0.5,3.800000,0.6,15.920058
Dice,0.920000,1.500000,1.008278,0.827080,0.830745,1.0,0.800000,1.0,0.090182
CCHVAE,0.450000,6.000000,1.409424,1.090609,0.650665,0.8,4.875000,0.0,0.085209
Revise,0.942857,5.428571,1.297401,0.965155,0.667741,0.7,6.142857,0.0,7.911986


In [19]:
from carla.data.catalog import CsvCatalog
continuous = ["Duration", "CreditAmount", "InstallmentRate", "ResidenceSince", "Age", "ExistingCredits", "NumPeople" ]
categorical = ["CheckingStatus", "CreditHistory", "Purpose", 
               "Savings", "Employment", "Personal", "OtherDebtors", "Property", 
               "OtherInstallmentPlans", "Housing", "Job", "Telephone", "ForeignWorker" ]
immutable = ["Personal", "Age"]
dataset = CsvCatalog(file_path="/Users/suzie/Downloads/statlog+german+credit+data/german_full.csv",
                     continuous=continuous,
                     categorical=categorical,
                     immutables=immutable,
                     target='Target')

In [20]:
dataset.df_test.sample(10)

,Duration,CreditAmount,InstallmentRate,ResidenceSince,Age,...,Job_skilled_employee,Job_unemployed_nonresident,Job_unskilled_resident,Telephone_yes,ForeignWorker_yes
889,0.352941,0.416749,0.666667,1.000000,0.375000,...,1.0,0.0,0.0,1.0,1.0
985,0.161765,0.065093,1.000000,0.666667,0.107143,...,1.0,0.0,0.0,0.0,1.0
344,0.088235,0.203533,0.000000,0.000000,0.321429,...,0.0,0.0,1.0,0.0,1.0
779,0.205882,0.199296,0.333333,1.000000,0.857143,...,1.0,0.0,0.0,1.0,1.0
267,0.294118,0.070595,1.000000,0.666667,0.339286,...,1.0,0.0,0.0,1.0,1.0
530,0.470588,0.111313,0.666667,0.000000,0.232143,...,1.0,0.0,0.0,0.0,1.0
957,0.073529,0.049741,0.333333,1.000000,0.321429,...,0.0,0.0,1.0,0.0,1.0
14,0.161765,0.063442,0.333333,1.000000,0.160714,...,1.0,0.0,0.0,0.0,1.0
950,0.205882,0.183779,0.666667,0.666667,0.375000,...,0.0,1.0,0.0,1.0,1.0
289,0.294118,0.042588,1.000000,1.000000,0.517857,...,1.0,0.0,0.0,0.0,1.0


In [27]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


X_train = dataset.df_train.drop(columns=[dataset.target])
y_train = dataset.df_train[dataset.target]


rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [29]:
from carla.models.api import MLModel

class CarlaRFModel(MLModel):
    def __init__(self, dataset, model):
        super().__init__(dataset)
        self._model = model

    @property
    def feature_input_order(self):
        # Important : cela doit être la liste d'input features, dans l'ordre attendu par le modèle
        # Tu peux faire dataset.df_train.columns.tolist(), sauf si tu as enlevé la colonne target
        return [col for col in self.data.df_train.columns if col != self.data.target]

    @property
    def backend(self):
        return "sklearn"

    @property
    def raw_model(self):
        return self._model

    def predict(self, x):
        # On ordonne les features comme attendu (important avec OHE)
        x_ord = self.get_ordered_features(x)
        return self._model.predict(x_ord)

    def predict_proba(self, x):
        x_ord = self.get_ordered_features(x)
        return self._model.predict_proba(x_ord)

In [30]:
import pandas as pd

from carla.models.catalog import MLModelCatalog
from carla.recourse_methods import GrowingSpheres, Wachter, Dice, CCHVAE, Revise
import carla.evaluation.catalog as evaluation_catalog
from carla.evaluation import Benchmark



mlmodel = CarlaRFModel(dataset, rf)  # ici ANN Pytorch

config_cchvae = {
    "data_name": "adult",
    "n_search_samples": 100,
    "p_norm": 1,
    "step": 0.1,
    "max_iter": 1000,
    "binary_cat_features": True,
    "vae_params": {
        "layers": [sum(mlmodel.get_mutable_mask()), 512, 256, 8],
        "train": True,
        "lambda_reg": 0.000001,
        "epochs": 5,
        "lr": 0.001,
        "batch_size": 32
    }
}

config_revise = {
    "data_name": "adult",
    "lambda": 0.5,
    "optimizer": "adam",
    "lr": 0.1,
    "max_iter": 1500,
    "target_class": [0, 1],
    "binary_cat_features": True,
    "vae_params": {
        "layers": [sum(mlmodel.get_mutable_mask()), 512, 256, 8],
        "activFun": None,
        "train": True,
        "lambda_reg": 0.000001,
        "epochs": 5,
        "lr": 0.001,
        "batch_size": 32
    }
}


recourse_methods = {
    "GrowingSpheres": GrowingSpheres(mlmodel),
    "Wachter": Wachter(mlmodel, {}),
    "Dice": Dice(mlmodel),
    "CCHVAE": CCHVAE(mlmodel, config_cchvae),
    "Revise": Revise(mlmodel, dataset, config_revise),
}
n_instances = 10
factuals = dataset.df.sample(10)



all_results = []

for name, recourse_method in recourse_methods.items():
    benchmark = Benchmark(mlmodel, recourse_method, factuals)
    metrics = [
    evaluation_catalog.YNN(mlmodel, {"y": 5, "cf_label": 1}),
    evaluation_catalog.Distance(mlmodel),
    evaluation_catalog.SuccessRate(),
    evaluation_catalog.Redundancy(mlmodel, {"cf_label": 1}),
    evaluation_catalog.ConstraintViolation(mlmodel),
    evaluation_catalog.AvgTime(recourse_method),  
    ]
    result = benchmark.run_benchmark(metrics)
    result["Recourse_Method"] = name
    all_results.append(result)

results_df = pd.concat(all_results, ignore_index=True)
results_df.set_index("Recourse_Method", inplace=True)
results_df


ValueError: sklearn is not in supported backends ['tensorflow', 'pytorch']